<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span><ul class="toc-item"><li><span><a href="#Linear-Regression" data-toc-modified-id="Linear-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Linear Regression</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

импортируем необходимые библиотеки и сохраним данные автомобилей сервиса «Не бит, не крашен»

In [282]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [283]:
data = pd.read_csv('/datasets/autos.csv')

выведем первые 10 строк таблицы:

In [284]:
data.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


In [285]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

в данных обнаружены пропуски. Больше всего в графе NotRepaired

для прогнозирования цены автомобиля нам не нужны признаки DateCrawled,DateCreated,NumberOfPictures,PostalCode и LastSeen. Удалим их

In [209]:
data=data.drop(columns=['DateCrawled','DateCreated','NumberOfPictures','PostalCode','LastSeen','RegistrationMonth'])

In [210]:
data.head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no
7,0,sedan,1980,manual,50,other,40000,petrol,volkswagen,no
8,14500,bus,2014,manual,125,c_max,30000,petrol,ford,NaN
9,999,small,1998,manual,101,golf,150000,NaN,volkswagen,NaN


In [211]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       316879 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           334536 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             334664 non-null  object
 6   Kilometer         354369 non-null  int64 
 7   FuelType          321474 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       283215 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


посмотрим сколько пропусков в каждом столбце:

In [212]:
data.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox             19833
Power                   0
Model               19705
Kilometer               0
FuelType            32895
Brand                   0
NotRepaired         71154
dtype: int64

начнем убирать пустоты и аномальные значения начиная с Price:

In [213]:
data['Price'].value_counts()

0        10772
500       5670
1500      5394
1000      4649
1200      4594
         ...  
13180        1
10879        1
2683         1
634          1
8188         1
Name: Price, Length: 3731, dtype: int64

Price это целевой признак и нулевые значения недопустимы. Уберем все значения с нулями

In [214]:
data=data.query('Price!=0')

проверим что все верно и нулей нет:

In [215]:
data['Price'].value_counts()

500      5670
1500     5394
1000     4649
1200     4594
2500     4438
         ... 
5240        1
13180       1
10879       1
2683        1
8188        1
Name: Price, Length: 3730, dtype: int64

посмотрим сколько данных пропущенно в VehicleType:

In [216]:
data['VehicleType'].isna().sum()/data.shape[0]

0.09822553747558914

пропущенно чуть меньше 10% данных. Заменим пропуски на unknown:

In [217]:
data['VehicleType']=data['VehicleType'].fillna('unknown')

и проверим что данные успешно изменились:

In [218]:
data['VehicleType'].value_counts()

sedan          89405
small          78018
wagon          63701
unknown        33750
bus            28345
convertible    19865
coupe          15636
suv            11791
other           3086
Name: VehicleType, dtype: int64

теперь посмотрим на RegistrationYear:

In [219]:
data['RegistrationYear'].value_counts()

2000    23072
1999    21995
2005    21524
2006    19679
2001    19654
        ...  
1949        1
2222        1
5300        1
8888        1
2290        1
Name: RegistrationYear, Length: 140, dtype: int64

видны странные значения, как 1200 или 4500 год. Ограничим данные по году регистрации автомобиля 1900 и 2023 годом.

In [220]:
data_new=data.query('RegistrationYear>1900 and RegistrationYear<2023')

посмотрим сколько процентов мы удалили

In [221]:
data_new.shape[0]/data.shape[0]*100

99.95954563049153

удалено меньше 1% 

In [222]:
data=data_new

проанализируем Gearbox

In [223]:
data['Gearbox'].value_counts()

manual    261351
auto       64864
Name: Gearbox, dtype: int64

тяжело отнести пропуски к какому-либо виду коробки передач, поэтому заменим их на неизвестные:

In [224]:
data['Gearbox']=data['Gearbox'].fillna('unknown')

In [225]:
data['Gearbox'].value_counts()

manual     261351
auto        64864
unknown     17243
Name: Gearbox, dtype: int64

проанализируем Power

In [226]:
data['Power'].value_counts()

0        36259
75       23317
60       15465
150      14170
101      12960
         ...  
1312         1
15001        1
1870         1
1436         1
1416         1
Name: Power, Length: 699, dtype: int64

обнаружено очень много нулей. Заменим их медианным значением.

In [227]:
data['Power']=data['Power'].replace(0,data['Power'].median())

In [228]:
data['Power'].value_counts()

105.0      46502
75.0       23317
60.0       15465
150.0      14170
101.0      12960
           ...  
10218.0        1
776.0          1
2172.0         1
1256.0         1
9010.0         1
Name: Power, Length: 698, dtype: int64

проанализируем Model

In [229]:
data['Model'].value_counts()

golf                  28233
other                 23690
3er                   19175
polo                  12625
corsa                 12191
                      ...  
i3                        8
rangerover                4
serie_3                   3
range_rover_evoque        2
serie_1                   1
Name: Model, Length: 250, dtype: int64

данные с неизвестной моделью не имеют смысла, поэтому удалим эти данные:

In [230]:
data=data[data['Model'].notna()]

проверим что пустот больше нет:

In [231]:
data.isna().sum()

Price                   0
VehicleType             0
RegistrationYear        0
Gearbox                 0
Power                   0
Model                   0
Kilometer               0
FuelType            23490
Brand                   0
NotRepaired         58218
dtype: int64

проанализируем Kilometer:

In [232]:
data['Kilometer'].value_counts()

150000    218579
125000     34387
100000     13753
90000      10998
80000       9562
70000       8170
60000       7110
50000       5921
40000       4659
5000        4227
30000       4120
20000       3520
10000        984
Name: Kilometer, dtype: int64

нулей и пустот в Kilometer не обнаружено, перейдем к FuelType:

In [233]:
data['FuelType'].value_counts()

petrol      202235
gasoline     94500
lpg           4845
cng            533
hybrid         204
other          118
electric        65
Name: FuelType, dtype: int64

аналогично изменим пропуски на other в виде топлива 

In [234]:
data['FuelType'] = data['FuelType'].fillna("unknown")

/tmp/ipykernel_49/4100503146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FuelType'] = data['FuelType'].fillna("unknown")


In [235]:
data['FuelType'].value_counts()

petrol      202235
gasoline     94500
unknown      23490
lpg           4845
cng            533
hybrid         204
other          118
electric        65
Name: FuelType, dtype: int64

In [236]:
data['NotRepaired'].value_counts()

no     235818
yes     31954
Name: NotRepaired, dtype: int64

In [237]:
data['NotRepaired']=data['NotRepaired'].fillna('unknown')

/tmp/ipykernel_49/2400037822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NotRepaired']=data['NotRepaired'].fillna('unknown')


проверим что в датасете больше нет пропусков:

In [238]:
data.isna().sum()

Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Kilometer           0
FuelType            0
Brand               0
NotRepaired         0
dtype: int64

проверим данные на явные дубликаты:

In [239]:
 data.duplicated().sum()

42440

дубликаты нам не нужны, удалим их:

In [240]:
data=data.drop_duplicates()

In [241]:
data.duplicated().sum()

0

преобразуем тип Power в целочисленный формат:

In [242]:
data['Power'] = data['Power'].apply(np.int64)

In [243]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283550 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             283550 non-null  int64 
 1   VehicleType       283550 non-null  object
 2   RegistrationYear  283550 non-null  int64 
 3   Gearbox           283550 non-null  object
 4   Power             283550 non-null  int64 
 5   Model             283550 non-null  object
 6   Kilometer         283550 non-null  int64 
 7   FuelType          283550 non-null  object
 8   Brand             283550 non-null  object
 9   NotRepaired       283550 non-null  object
dtypes: int64(4), object(6)
memory usage: 23.8+ MB


В наборе данных встречаются категориальные признаки. Чтобы преобразовать категориальные признаки в численные воспользуемся техникой One-Hot Encoding:

In [244]:
data.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
0,480,unknown,1993,manual,105,golf,150000,petrol,volkswagen,unknown
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes


In [245]:
data_ohe = pd.get_dummies(data, drop_first=True)

In [246]:
data_ohe.head()

,Price,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_unknown,NotRepaired_yes
0,480,1993,105,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,9800,2004,163,125000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1500,2001,75,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,3600,2008,69,90000,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
5,650,1995,102,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


перейдем к обучению моделей.

## Обучение моделей

разобьем исходные данные на обучающую выборку, валидационную выборку и тестовую выборку в соотношении 3:1:1. Для этого сначала разделим данные в соотношении 3:2, а затем вторую часть разделим в отношении 1:1

In [247]:
data_train,data_validtest = train_test_split(data_ohe, test_size=0.4, random_state=42)

data_valid,data_test=train_test_split(data_validtest, test_size=0.5, random_state=42)

проверим что данные поделились верно:

In [248]:
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(170130, 310)
(56710, 310)
(56710, 310)


выделим признаки:

In [249]:
features_train = data_train.drop(['Price'], axis=1)

и целевой признак:

In [250]:
target_train = data_train['Price']

и аналоигчно для валидационной и тестовой выборок:

In [251]:
features_valid = data_valid.drop(['Price'], axis=1)
target_valid = data_valid['Price']

features_test = data_test.drop(['Price'], axis=1)
target_test = data_test['Price']

чтобы маштабировать признаки запишем все численные признаки в список numeric

In [252]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']

и создадим объект scaler и обучим его на обучающих данных.

In [253]:
scaler = StandardScaler()
scaler.fit(features_train[numeric]) 

StandardScaler()

и сохраним обучающие и тестирующие выборки:

In [254]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric]) 

## Анализ моделей

обучим модель линейной регрессии:

### Linear Regression

In [255]:
%%time
LinearRegression_model = LinearRegression()
LinearRegression_model.fit(features_train, target_train)

CPU times: user 8.28 s, sys: 3.87 s, total: 12.2 s
Wall time: 12.1 s


LinearRegression()

In [256]:
%%time
predictions = LinearRegression_model.predict(features_valid)
  
rmse = mean_squared_error(target_valid,predictions,squared=False)
print(rmse)

2839.0347191871165
CPU times: user 75 ms, sys: 52.5 ms, total: 128 ms
Wall time: 143 ms


обучим модель LightGBM:

### LightGBM

обучим LightGBM c числом деревьев 10 и максимальным значением базовых "листьев" 10

In [257]:
%%time
LightGBM_model1 = LGBMRegressor(n_estimators=10,num_leaves=10,random_state=42)
LightGBM_model1.fit(features_train, target_train)

CPU times: user 1.92 s, sys: 165 ms, total: 2.09 s
Wall time: 2.1 s


LGBMRegressor(n_estimators=10, num_leaves=10, random_state=42)

In [258]:
%%time
predictions = LightGBM_model1.predict(features_valid)
  
rmse = mean_squared_error(target_valid,predictions,squared=False)
print(rmse)

2912.4098058108752
CPU times: user 190 ms, sys: 47.9 ms, total: 238 ms
Wall time: 295 ms


теперь увеличим число деревьев и максимальное значение листьев до 100

In [259]:
%%time
LightGBM_model2 = LGBMRegressor(n_estimators=100,num_leaves=100,random_state=42)
LightGBM_model2.fit(features_train, target_train)

CPU times: user 8.72 s, sys: 111 ms, total: 8.83 s
Wall time: 8.89 s


LGBMRegressor(num_leaves=100, random_state=42)

In [260]:
%%time
predictions = LightGBM_model2.predict(features_valid)
  
rmse = mean_squared_error(target_valid,predictions,squared=False)
print(rmse)

1664.4990631556313
CPU times: user 917 ms, sys: 35.5 ms, total: 953 ms
Wall time: 1e+03 ms


метрика rmse заметно улушилась, что логично, но время увеличилось в 4 раза.

### CatBoost

In [269]:
%%time
CatBoost_model3 = CatBoostRegressor(iterations=350,learning_rate=0.5, random_state=42, verbose=50) 
CatBoost_model3.fit(features_train, target_train)

0:	learn: 3300.4499717	total: 26.2ms	remaining: 9.15s
50:	learn: 1774.1631458	total: 1.08s	remaining: 6.31s
100:	learn: 1680.5229083	total: 2.16s	remaining: 5.33s
150:	learn: 1632.7975729	total: 3.18s	remaining: 4.19s
200:	learn: 1602.6641963	total: 4.19s	remaining: 3.11s
250:	learn: 1574.3459467	total: 5.24s	remaining: 2.07s
300:	learn: 1551.0522321	total: 6.26s	remaining: 1.02s
349:	learn: 1531.5864546	total: 7.25s	remaining: 0us
CPU times: user 8.23 s, sys: 76.3 ms, total: 8.3 s
Wall time: 8.86 s


In [270]:
%%time
predictions = CatBoost_model3.predict(features_valid)
  
rmse = mean_squared_error(target_valid,predictions,squared=False)
print(rmse)

1660.4754230377525
CPU times: user 58.1 ms, sys: 312 µs, total: 58.4 ms
Wall time: 56.8 ms


попробуем увеличить число иттераций и уменьшить learning_rate

In [263]:
%%time
CatBoost_model3 = CatBoostRegressor(iterations=600,learning_rate=0.1, random_state=42, verbose=50) 
CatBoost_model3.fit(features_train, target_train)

0:	learn: 4296.2066356	total: 27.5ms	remaining: 13.7s
50:	learn: 1995.9947217	total: 1.13s	remaining: 9.94s
100:	learn: 1867.2100023	total: 2.13s	remaining: 8.43s
150:	learn: 1809.8786881	total: 3.16s	remaining: 7.3s
200:	learn: 1772.0021484	total: 4.19s	remaining: 6.23s
250:	learn: 1743.5683332	total: 5.19s	remaining: 5.15s
300:	learn: 1721.7010518	total: 6.18s	remaining: 4.08s
350:	learn: 1702.6129093	total: 7.17s	remaining: 3.04s
400:	learn: 1686.4245336	total: 8.17s	remaining: 2.02s
450:	learn: 1672.1176252	total: 9.21s	remaining: 1s
499:	learn: 1661.3026493	total: 10.2s	remaining: 0us
CPU times: user 11.1 s, sys: 79.8 ms, total: 11.2 s
Wall time: 11.8 s


In [264]:
%%time
predictions = CatBoost_model3.predict(features_valid)
  
rmse = mean_squared_error(target_valid,predictions,squared=False)
print(rmse)

1704.0024897283697
CPU times: user 58.8 ms, sys: 326 µs, total: 59.1 ms
Wall time: 57.8 ms


In [271]:
df =   [["12.1 s", "143 ms", 2839],
        ["2.1 ms", "295 ms", 2912],
        ["8.89 s", "0.001  ms", 1664],
        ["8.86 s", "56.8 ms", 1660],
        ["11.8 s", "57.8 ms", 1704]]
model = ["Linear Regression", "LightGBM", "LightGBM", "CatBoostRegressor", "CatBoostRegressor"]

In [272]:
pd.DataFrame(data=df, index=model, columns=["время обучения", "время предсказания", "RMSE"])

,время обучения,время предсказания,RMSE
Linear Regression,12.1 s,143 ms,2839
LightGBM,2.1 ms,295 ms,2912
LightGBM,8.89 s,0.001 ms,1664
CatBoostRegressor,8.86 s,56.8 ms,1660
CatBoostRegressor,11.8 s,57.8 ms,1704


## Вывод

Самое мальнькое значение RMSE при наименьшем времени обучения показала модель CatBoostRegressor. Выберем ее и проверим работу на тестовой выборке:

In [280]:
CatBoost_model3 = CatBoostRegressor(iterations=350,learning_rate=0.5, random_state=42, verbose=50) 
CatBoost_model3.fit(features_train, target_train)

predictions = CatBoost_model3.predict(features_test)
  
rmse = mean_squared_error(target_test,predictions,squared=False)
print(rmse)

0:	learn: 3300.4499717	total: 36.9ms	remaining: 12.9s
50:	learn: 1774.1631458	total: 1.13s	remaining: 6.6s
100:	learn: 1680.5229083	total: 2.19s	remaining: 5.39s
150:	learn: 1632.7975729	total: 3.24s	remaining: 4.28s
200:	learn: 1602.6641963	total: 4.26s	remaining: 3.16s
250:	learn: 1574.3459467	total: 5.33s	remaining: 2.1s
300:	learn: 1551.0522321	total: 6.51s	remaining: 1.06s
349:	learn: 1531.5864546	total: 7.6s	remaining: 0us
1666.7044346880639
